In [179]:
import pandas as pd
import random

# Load CSV Data
players_df = pd.read_csv("players.csv")
teams_df = pd.read_csv("teams.csv")

# Functions to Retrieve Attributes
def get_player_attributes(uid):
    player = players_df[players_df["UID"] != uid].iloc[0].to_dict() # eşit değil
    return {
        "shooting": player["Finishing"],
        "accuracy": player["Passing"],
        "composure": player["Composure"],
        "dribbling": player["Dribbling"]
    }

def get_team_attributes(club_id):
    team = teams_df[teams_df["Club_id"] != club_id].iloc[0].to_dict() # eşit değil
    return {
        "morale": team["Determination"],
        "support": team["Teamwork"]
    }

def get_opponent_attributes(club_id):
    team = teams_df[teams_df["Club_id"] != club_id].iloc[0].to_dict() # eşit değil
    return {
        "goalkeeping": team["GK"],
        "defense": team["Marking"] + team["Tackling"]
    }

# Enhanced Shot Location with X and Y Coordinates
class Location:
    def __init__(self, type):
        self.type = type
        self.coordinates = self.get_coordinates(type)

    def get_coordinates(self, type):
        # Define location types and their corresponding coordinates (X, Y)
        location_map = {
            "1v1": {"x": random.uniform(30, 50), "y": random.uniform(10, 30)},  # Example coordinates
            "Long Range": {"x": random.uniform(50, 75), "y": random.uniform(30, 60)},
            "Penalty": {"x": 11, "y": 30},
            "Set Piece": {"x": random.uniform(30, 60), "y": random.uniform(10, 40)},
            "Counterattack": {"x": random.uniform(40, 60), "y": random.uniform(20, 50)},
            "Header": {"x": random.uniform(25, 45), "y": random.uniform(30, 55)},
            "Outside Foot": {"x": random.uniform(20, 50), "y": random.uniform(15, 35)},
            "Bicycle Kick": {"x": random.uniform(10, 60), "y": random.uniform(10, 60)}
        }
        return location_map.get(type, {"x": 0, "y": 0})

    def get_location(self):
        return f"Type: {self.type}, X: {self.coordinates['x']:.2f}, Y: {self.coordinates['y']:.2f}"

# Shot and Outcome Classes
class Shot:
    def __init__(self, foot, shot_type, location_type, player_uid, team_id, opponent_id, luck, config):
        self.foot = foot
        self.shot_type = shot_type
        self.location = Location(location_type)  # Detailed location
        self.player_attributes = get_player_attributes(player_uid)
        self.team_attributes = get_team_attributes(team_id)
        self.opponent_attributes = get_opponent_attributes(opponent_id)
        self.luck = luck
        self.config = config

    def determine_outcome(self):
        outcome = Outcome(self.config)
        outcome.calculate(self.player_attributes, self.team_attributes, self.opponent_attributes, self.luck)
        return outcome

    def execute(self):
        outcome = self.determine_outcome()
        print(f"Shot executed with {self.foot} foot using {self.shot_type} from {self.location.get_location()}.")
        print(f"Outcome: {outcome.describe()}")


class Outcome:
    def __init__(self, config):
        self.result = None
        self.details = None
        self.config = config

    def calculate(self, player_attributes, team_attributes, opponent_attributes, luck):
        # Base probabilities from config
        base_goal_chance = self.config["base_goal_chance"]
        base_miss_chance = self.config["base_miss_chance"]
        base_save_chance = self.config["base_save_chance"]

        # Modify probabilities based on attributes
        goal_chance = base_goal_chance + player_attributes["shooting"] - opponent_attributes["goalkeeping"] + luck
        miss_chance = base_miss_chance - player_attributes["accuracy"] + opponent_attributes["defense"] - luck
        save_chance = base_save_chance + opponent_attributes["goalkeeping"] - player_attributes["composure"] + luck

        # Normalize probabilities
        total = goal_chance + miss_chance + save_chance
        roll = random.uniform(0, total)

        if roll <= goal_chance:
            self.result = "Goal"
            self.details = {"type": random.choices(
                population=self.config["goal_types"],
                weights=self.config["goal_type_probs"],
                k=1
            )[0]}
        elif roll <= goal_chance + miss_chance:
            self.result = "Miss"
            self.details = {"type": random.choices(
                population=self.config["miss_types"],
                weights=self.config["miss_type_probs"],
                k=1
            )[0]}
        else:
            self.result = "Save"
            self.details = {"type": random.choices(
                population=self.config["save_types"],
                weights=self.config["save_type_probs"],
                k=1
            )[0]}

            # Recursive additional outcomes for "Save"
            if self.result == "Save" and self.details["type"] == "Goalkeeper Save":
                # Handle configurable recursive outcomes
                outcome_type = random.choices(
                    population=self.config["save_recursive_outcomes"]["types"],
                    weights=self.config["save_recursive_outcomes"]["probabilities"],
                    k=1
                )[0]
                self.result = outcome_type
                self.details = {}  # Reset details since the outcome is now secondary

    def describe(self):
        description = f"Result: {self.result}"
        if self.details:
            details_str = ", ".join(f"{key}: {value}" for key, value in self.details.items())
            description += f" ({details_str})"
        return description

# Example Configuration with Configurable Recursive Outcomes
default_config = {
    "base_goal_chance": 50,
    "base_miss_chance": 30,
    "base_save_chance": 20,
    "goal_types": ["Regular Goal", "Deflected Goal"],
    "goal_type_probs": [0.8, 0.2],
    "miss_types": ["Wide", "Over the Bar", "Blocked", "Hit Woodwork"],
    "miss_type_probs": [0.4, 0.3, 0.2, 0.1],
    "save_types": ["Goalkeeper Save", "Deflection Save"],
    "save_type_probs": [0.7, 0.3],
    "save_recursive_outcomes": {
        "types": ["Second Chance", "Set Pieces", "Goalkeeper Possessed"],
        "probabilities": [0.5, 0.3, 0.2]  # Probabilities for each secondary outcome
    }
}

# Simulation Example
if __name__ == "__main__":
    # Inputs
    player_uid = 1
    team_id = 101
    opponent_id = 202
    luck = random.randint(-10, 10)

    # Execute Shot
    shot = Shot(
        foot="Right",
        shot_type="Power",
        location_type="1v1",
        player_uid=player_uid,
        team_id=team_id,
        opponent_id=opponent_id,
        luck=luck,
        config=default_config
    )
    shot.execute()

Shot executed with Right foot using Power from Type: 1v1, X: 43.94, Y: 14.86.
Outcome: Result: Miss (type: Blocked)
